<a href="https://colab.research.google.com/github/B10956048/113_Finance/blob/main/week2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1.證券交易所API

In [ ]:
import requests
import pandas as pd
from datetime import date

In [ ]:
# prompt: 使用此API，https://www.twse.com.tw/rwd/zh/afterTrading/STOCK_DAY?date={date}&stockNo={stock_id}') date為今日 stock_id = 2454
today = date.today().strftime("%Y%m%d")
stock_id = '2454'
url = f'https://www.twse.com.tw/rwd/zh/afterTrading/STOCK_DAY?date={today}&stockNo={stock_id}'

response = requests.get(url)
if response.status_code == 200:
  data = response.json()
  if 'data' in data:
      df = pd.DataFrame(data['data'], columns=data['fields'])
  else:
      print('No data found in the response.')
else:
  print(f'Failed to retrieve data. Status code: {response.status_code}')


In [ ]:
df

,日期,成交股數,成交金額,開盤價,最高價,最低價,收盤價,漲跌價差,成交筆數
0,113/09/02,"3,294,144","4,033,872,708","1,240.00","1,250.00","1,215.00","1,225.00",-15.00,"5,675"
1,113/09/03,"3,534,979","4,292,070,390","1,225.00","1,225.00","1,200.00","1,220.00",-5.00,"6,693"
2,113/09/04,"10,457,652","11,943,320,285","1,135.00","1,170.00","1,130.00","1,135.00",-85.00,"31,959"
3,113/09/05,"9,432,513","10,509,325,015","1,120.00","1,130.00","1,105.00","1,110.00",-25.00,"21,502"
4,113/09/06,"5,340,953","6,061,653,645","1,110.00","1,150.00","1,105.00","1,145.00",+35.00,"7,743"
5,113/09/09,"7,455,495","8,209,712,700","1,100.00","1,115.00","1,090.00","1,105.00",-40.00,"17,956"
6,113/09/10,"5,893,397","6,599,039,331","1,115.00","1,130.00","1,100.00","1,130.00",+25.00,"8,084"
7,113/09/11,"4,096,499","4,588,000,025","1,125.00","1,135.00","1,110.00","1,110.00",-20.00,"6,026"
8,113/09/12,"4,719,178","5,409,189,304","1,150.00","1,155.00","1,135.00","1,155.00",+45.00,"6,893"
9,113/09/13,"3,546,828","4,032,559,340","1,160.00","1,165.00","1,125.00","1,125.00",-30.00,"6,532"


# 2.YAHOO爬蟲

In [ ]:
from datetime import date
from bs4 import BeautifulSoup
import pandas as pd

In [ ]:
url = "https://tw.stock.yahoo.com/quote/2454.TW"

response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

stock_name = soup.find('h1', class_='C($c-link-text) Fw(b) Fz(24px) Mend(8px)').get_text()
close_time = soup.find('span', class_="C(#6e7780) Fz(12px) Fw(b)").get_text()
price_details = soup.find_all('li', class_='price-detail-item')

data = {}

for item in price_details:
    spans = item.find_all('span')  # Get all spans within the 'li' tag

    label = ""
    value = ""
    for i, span in enumerate(spans):
        if span.text == '':
            continue
        if i == 0:
            label = span.text  # label
        elif i == 1:
            value = span.text  # value
    if label and value:
        data[label] = [value]
df = pd.DataFrame(data)
df.insert(0, '股票名稱', stock_name)
df.insert(1, '收盤時間', close_time)

In [ ]:
df

,股票名稱,收盤時間,成交,開盤,最高,最低,均價,成交金額(億),昨收,漲跌幅,漲跌,總量,昨量,振幅
0,聯發科,收盤 | 2024/09/23 14:30 更新,"1,145","1,125","1,150","1,120","1,138",51.09,"1,125",1.78%,20.00,"4,487","8,868",2.67%
